### API Streaming do Twitter

#### API streaming do Twitter nos permite ter acesso contínuo de novos tweets. É muito usado para rastrear um determinado assunto ou tópico específico. Para tanto utilizaremos uma instância da classe tweepy.Stream.

Para termos acesso a essas funcionalidades precisamos de credenciais do Twitter. Com um conta criada é possível ver as credenciais em:

>[developer.twitter.com](https://developer.twitter.com/) -> ‘Usuário‘ -> Apps -> Details -> Keys and tokens

É necessário buscar:
*	Consumer Key
*	Consumer Secret
*	Acces Token
*	Acces Token Secret



In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [ ]:
from time import time
import tweepy

In [ ]:
autorizar = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [ ]:
autorizar.set_access_token(access_token, access_token_secret)

#### Com o módulo Tweepy podemos acessar a API do Twitter

In [ ]:
api = tweepy.API(autorizar)

In [ ]:
print(api)

#### Usaremos uma hierarquia de classes e colocaremos o método on_data que irá receber todas as mensagens.

In [ ]:
class Dados_Twitter(tweepy.StreamListener):
    
    def __init__(self, nome_arq, limite):
        self.tempo_inicial = time()
        self.limite = limite
        self.salvar_arquivos = open(nome_arq, mode='a', newline='')
# A opção de usar o método newline=’ ‘ é para que o arquivo tenho separação entre os tweets salvos.    
    def on_data(self, dados):
        if (time() - self.tempo_inicial < self.limite):
            self.salvar_arquivos.write(dados)
            return True
        else:
            self.salvar_arquivos.close()
            return False

Para termos um tempo determinado de captação de tweets usamos time() para saber o tempo inicial e fazemos um laço para iterar com o tempo corrente. Comparamos com o tempo limite para pararmos a captação. 

Os dados recuperados pela API estão no formato JSON. Precisamos criar um arquivo (tweets_60s.json) para receber esses dados. Para isso utilizaremos o open com mode=’a’. Esse modo está descrito como: “open for writing, appending to the end of the file if it exists”. 

In [ ]:
nome_arq = 'tweets_60s.json'
dados_twitter = Dados_Twitter(nome_arq, 60)

Finalmente, usamos o filter() para selecionarmos os termos que queremos buscar. Com o track montamos um array de termos que gostaríamos.

In [ ]:
fluxo = tweepy.Stream(autorizar, dados_twitter)

fluxo.filter(track=['Neymar'], languages=['pt'])

Salvamos um arquivo que recebe dados no formato json.

In [ ]:
import json
import pandas as pd

dados = []

#Com o with podemos garantir finalização de recursos adquiridos.
#Nesse caso garantindo que o arquivo será aberto e fechado.
with open('tweets_60s.json') as arquivo:
    for linha in arquivo:
        dados.append(json.loads(linha))

In [ ]:
#Chamando dados[0] podemos ver a estrutura de um tweet.

dados[0]

Colocamos então nossa captura de 60 segundos em um dataframe do pandas.

In [ ]:
df = pd.DataFrame(dados)

In [ ]:
print(df.columns)

In [ ]:
df.head()

In [ ]:
len(df)

Queremos buscar de cada tweet: text, created_at, screen_name, followers_count.

In [ ]:
print(df['text'][0])
print(df['created_at'][0])
print(df['user'][0]['screen_name'])
print(df['user'][0]['followers_count'])

Vamos criar uma lista justamente com o queremos dos tweets.

In [ ]:
colunas = ['text', 'created_at', 'screen_name', 'followers_count']

In [ ]:
#Vamos criar um dataframe auxiliar para salvar somente os dados que queremos.

df_aux = pd.DataFrame(columns=colunas)
df_aux

In [ ]:
for i in range(0, len(df)):
    dados = [
        df['text'][i],
        df['created_at'][i],
        df['user'][i]['screen_name'],
        df['user'][i]['followers_count'],
        ]
    series = pd.Series(dados,index=colunas)
    df_aux = df_aux.append(series, ignore_index=True)

In [ ]:
print(df_aux)

Por fim, salvamos o arquivo com as informações que queremos

In [ ]:
#df_aux.to_csv('tweets_60s.csv', sep=';', encoding='utf-8', index=False)

https://tweepy.readthedocs.io/en/latest/streaming_how_to.html